# GWO DailyまたはHoulyの複数測点csvを測点別csvに分割する
**Author: Jun Sasaki**  **Coded on 2018-09-16, revised on 2024-07-02**<br>
- 気象データベース地上観測（2018年販売終了，関連情報は[ウェザートーイ](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/)）DVDとサポートデータを処理するためのスクリプトである．
- データベースは主に気象官署データ（GWO）とアメダスデータ（AMD）から構成されている．
- 購入・登録者限定で配布されている，SqlView7.exe を用いると，データベースから全観測所の全期間を含む，任意の条件でCSVファイルへの切り出しが可能である．このスクリプトは基本的に全観測所，全期間を含む一つのCSVファイルを観測所毎のフォルダに観測年別CSVファイルとして分割出力するためのスクリプトである． 

## 全体の流れ
- このNotebookで，SqlView7.exeを用いて切り出した全観測点CSVファイルを読み込み，観測点別のCSVファイルとして指定されたディレクトリに出力する．
- この出力された観測点別のCSVファイルをGWO_div_year.ipynb で読み込み，観測点別ディレクトリに年別CSVファイルとして出力する．

### GWO 気象官署時別値・日別値
時別値は1961年から1990年までは3時間間隔データとなっている．GWOのDVDから1地点1ファイルとして複数年一括出力したものを年毎に分ける．1991年以降は1時間間隔となっている．本コードはどちらにも，あるいは混在していても適用可能である．このほかに日別値にも同様に対応した．

### 注意
- 入力csvファイルのエンコードはSHIFT-JIS，CRLF，分割後の観測点別ファイルのエンコードもSHIFT-JIS，CRLFである．
- データベースはSqlViewe7を立ち上げ，「有効データ」および「閾値ソート」のチェックを外し，「全データベース項目」にチェックを入れ，全観測所を選び，全期間を指定して，CSV出力として実行する．なお，GWO時別値は測定時間間隔が異なるため，1990年以前と1991年以降に分割した．
- 時別値は1日の最後のデータ時刻が24時のため，年末24時のデータは翌年年初0時のデータを意味する．
- 将来的にはNetCDFで整理するのがよさそう．

# Split GWO Daily or Houly multi-year and multi-station csv file into each year and station csv files
- Meteorological Database Ground Observation product by JMA (only in Japanese, discontinued in 2018, related information is available at [Weathertoy](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/)).
- This is a script for processing the DVD and supporting data. The database consists mainly of Japan Meteorological Agency observatory data (GWO) and AMeDAS data (AMD).
- **SqlView7.exe**, available only to registered users, can be used to extract CSV files from the database under any conditions, including all periods for all stations.
- This script is basically to split a CSV file containing all stations and all periods into separate CSV files for each year and station in each station's folder.

## Workflow
- In this Notebook, read the CSV files of all observation stations cut out by SqlView7.exe and output them as a CSV file for each observation station in a specified directory.
- Teses output CSV filsearesto be  read by** GWO_div_year.ipyn**b and output as CSV filse by year ineach of the eobservation station  directoiest.

### GWO Hourly and Daily Meteorological Data
The input data files are three-hour-interval data from 1961 to 1990, and one-hour-interval data from 1991 onwards. This code can be applied to either of them or both of them at once.

### Note
- The encoding of the input csv file is SHIFT-JIS and CRLF, and the encoding of the split file is also SHIFT-JIS and CRLF.
- For the database, start SqlViewe7, uncheck "valid data" and "threshold sort", check "all database items", select all stations, specify all time periods, and run the CSV output. Note that the GWO hourly values are divided into pre-1990 and post-1991 because of the different measurement time intervals.
- Since the last data time of the day is 24:00, the data at 24:00 at the end of the year means the data at 0:00 at the beginning of the next year.
- In the future, it would be better to use NetCDF to organize the data.

In [ ]:
import pandas as pd
import os
import sys

## stn_dict 辞書を作成する
観測所名（漢字）を自動認識し，その測点名に対応するディレクトリ名をstn_dictから得る
## Create the stn_dict dicrionary.
Automatically recognize a station name (in Chinese characters) and get the directory name corresponding to the station name from stn_dict.

In [ ]:
stn_dict={"稚内":"Wakkanai", "北見枝幸":"Kitamiesashi", "羽幌":"Haboro", "雄武":"Oumu", "留萌":"Rumoi", "旭川":"Asahikawa", \
          "網走":"Abashiri", "小樽":"Otaru", "札幌":"Sapporo", "岩見沢":"Iwamizawa", "帯広":"Obihiro", "釧路":"Kushiro", \
          "根室":"Nemuro", "寿都":"Suttu", "室蘭":"Muroran", "苫小牧":"Tomakomai", \
          "浦河":"Urakawa", "江差":"Esashi", "函館":"Hakodate", "倶知安":"Kutchan", "紋別":"Monbetsu", "広尾":"Hiroo", \
          "大船渡":"Ofunato", "新庄":"Shinjo", "若松":"Wakamatsu", "深浦":"Fukaura", \
          "青森":"Aomori", "むつ":"Mutsu", "八戸":"Hachinohe", "秋田":"Akita", "盛岡":"Morioka", "宮古":"Miyako", \
          "酒田":"Sakata", "山形":"Yamagata", "仙台":"Sendai", "石巻":"Ishinomaki", \
          "福島":"Fukushima", "白河":"Shirakawa", "小名浜":"Onahama", "輪島":"Wajima", "相川":"Aikawa", "新潟":"Niigata", \
          "金沢":"Kanazawa", "伏木":"Fushiki", "富山":"Toyama", "長野":"Nagano", \
          "高田":"Takada", "宇都宮":"Utsunomiya", "福井":"Fukui", "高山":"Takayama", "松本":"Matsumoto", \
          "諏訪":"Suwa", "軽井沢":"Karuizawa", "前橋":"Maebashi", "熊谷":"Kumagaya", "水戸":"Mito", \
          "敦賀":"Tsuruga", "岐阜":"Gifu", "名古屋":"Nagoya", "飯田":"Iida", "甲府":"Kofu", \
          "河口湖":"Kawaguchiko", "秩父":"Chichibu", "館野":"Tateno", "銚子":"Choshi", "上野":"Ueno", \
          "津":"Tsu", "伊良湖":"Irago", "浜松":"Hamamatsu", "御前崎":"Omaezaki", "静岡":"Shizuoka", \
          "三島":"Mishima", "東京":"Tokyo", "尾鷲":"Owase", "石廊崎":"Irozaki", "網代":"Ajiro", \
          "横浜":"Yokohama", "館山":"Tateyama", "勝浦":"Katsuura", "大島":"Oshima", "三宅島":"Miyakejima", \
          "八丈島":"Hachijojima", "千葉":"Chiba", "四日市":"Yokkaichi", "日光":"Nikko", "西郷":"Saigo", \
          "松江":"Matsue", "境":"Sakai", "米子":"Yonago", "鳥取":"Tottori", "豊岡":"Toyooka", "舞鶴":"Maiduru", \
          "伊吹山":"Ibukiyama", "萩":"Hagi", "浜田":"Hamada", "津山":"Tsuyama", \
          "京都":"Kyoto", "彦根":"Hikone", "下関":"Shimonoseki", "広島":"Hiroshima", "呉":"Kure", \
          "福山":"Fukuyama", "岡山":"Okayama", "姫路":"Himeji", "神戸":"Kobe", "大阪":"Osaka", \
          "洲本":"Sumoto", "和歌山":"Wakayama", "潮岬":"Shionomisaki", "奈良":"Nara", "山口":"Yamaguchi", \
          "厳原":"Izuhara", "平戸":"Hirado", "福岡":"Fukuoka", "飯塚":"Iiduka", "佐世保":"Sasebo", \
          "佐賀":"Saga", "日田":"Hita", "大分":"Oita", "長崎":"Nagasaki", "熊本":"Kumamoto", \
          "阿蘇山":"Asosan", "延岡":"Nobeoka", "阿久根":"Akune", "人吉":"Hitoyoshi", "鹿児島":"Kagoshima", \
          "都城":"Miyakonojo", "宮崎":"Miyazaki", "枕崎":"Makurazaki", "油津":"Aburatsu", "屋久島":"Yakushima", \
          "種子島":"Tanegashima", "牛深":"Ushibuka", "福江":"Fukue", "松山":"Matsuyama", "多度津":"Tadotsu", \
          "高松":"Takamatsu", "宇和島":"Uwajima", "高知":"Kochi", "剣山":"Tsurugisan", "徳島":"Tokushima", \
          "宿毛":"Sukumo", "清水":"Shimizu", "室戸岬":"Murotomisaki", "名瀬":"Nase", "与那国島":"Yonakunijima", \
          "石垣島":"Ishigakijima", "宮古島":"Miyakojima", "久米島":"Kumejima", "那覇":"Naha", "名護":"Nago", \
          "沖永良部":"Okinoerabu", "南大東島":"Minamidaitojima", "父島":"Chichijima", "南鳥島":"Minamitorishima"}

## Define a function that divides a multi-year-multi-station file into each station file with multi-year

In [ ]:
def GWO_stns2stn(fname, db_path, stn_path, hourly=True, overwrite=False):
    '''Divide a file containing multiple stations csv to each station csv'''

    if hourly:
        names = ["KanID","Kname","KanID_1","YYYY","MM","DD","HH","lhpa","lhpaRMK","shpa","shpaRMK","kion","kionRMK","stem",\
                 "stemRMK","rhum","rhumRMK","muki","mukiRMK","sped","spedRMK","clod","clodRMK","tnki","tnkiRMK","humd","humdRMK",\
                 "lght","lghtRMK","slht","slhtRMK","kous","kousRMK"]
    else:
        names = ["KanID","Kname","KanID_1","YYYY","MM","DD","avrLhpa","avrLhpaRMK","avrShpa","avrShpaRMK","minShpa","minShpaRMK",\
                 "avrKion","avrKionRMK","maxKion","maxKionRMK","minKion","minKionRMK","avrStem","avrStemRMK","avrRhum","avrRhumRMK",\
                  "minRhum","minRhumRMK","avrSped","avrSpedRMK","maxSped","maxSpedRMK","maxMuki","maxMukiRMK","maxSSpd","maxSSpdRMK",\
                  "maxSMuk","maxSMukRMK","avrClod","avrClodRMK","daylght","daylghtRMK","sunlght","sunlghtRMK","amtEva","amtEvaRMK",\
                  "dayPrec","dayPrecRMK","maxHPrc","maxHPrcRMK","maxMPrc","maxMPrcRMK","talSnow","talSnowRMK","daySnow","daySnowRMK",\
                  "tenki1","tenki1RMK","tenki2","tenki2RMK","apCode1","apCode2","apCode3","apCode4","apCode5","strgTim","strgTimRMK"]
    
    input_fpath="{}/GWO/{}.csv".format(db_path, fname)
    print("Reading ", input_fpath)
    df = pd.read_csv(input_fpath, header=None, names=names, dtype="str", encoding="SHIFT-JIS")
    for stn in set(df['Kname']):
        print(stn)
        df_stn = df[df['Kname']==stn]
        
        dirpath_stn = "{}/GWO/{}/{}".format(db_path, stn_path ,stn_dict[stn])
        fpath_stn = "{}/{}{}.csv".format(dirpath_stn, stn_dict[stn],fname[-9:])
        #if hourly:
        #    fpath_stn = "{}/{}{}.csv".format(dirpath_stn, stn_dict[stn], fname[-9:])
        #else:
        #    fpath_stn = dirpath_stn + "/" + stn_dict[stn] + "1961-2017" + ".csv"  ### output CSV file path for each year
        if not os.path.isdir(dirpath_stn):  ### directoryが存在しない場合は作成
            print("Creating directory of " + dirpath_stn)
            os.makedirs(dirpath_stn)
        if not os.path.isfile(fpath_stn):
            print("Creating " + fpath_stn)
            df_stn.to_csv(fpath_stn, header=None, index=False, encoding="SHIFT-JIS")  ### エンコードをSHIFT-JISとする
        elif overwrite:
            print("Overwriting " + fpath_stn)
            df_stn.to_csv(fpath_stn, header=None, index=False, encoding="SHIFT-JIS")  ### エンコードをSHIFT-JISとする
        else:
            print("Already existing " + fpath_stn)

# ファイル分割の実行
# Start division of file
### Set database directory path and input file names

In [ ]:
# JMA DataBase directory path (without "/")
db_path = "/mnt/d/dat/met/JMA_DataBase"
# CSV file names without extention: Daily and two hourly files.
daily_file="GWODaily1961-2021"
hourly_file_bf1990 = "GWOHourly1961-1990"
hourly_file_af1991 = "GWOHourly1991-2021"
# Output directory names for daily and hourly data under db_path (without "/")
dir_daily = "Daily2"
dir_hourly = "Hourly3"

### GWO Daily data
- アーカイブデータファイル名を与え，観測点別のファイルに分割する．観測点名はファイル中の漢字名からstn_dictによって自動設定される．
- The archive data file name is given and divided into files for each observation station. The observation station name is automatically set by `stn_dict` from the Chinese characters' name in the file.

In [ ]:
GWO_stns2stn(fname=daily_file, db_path=db_path, stn_path=dir_daily, hourly=False, overwrite=False)

### GWO Hourly data
- 1961-1990のGWOHourly1961-1990.csvと1991-2021のGWOHourly1991-2021.csvをそれぞれ，観測点毎のファイルに分割する．
- Split GWOHourly1961-1990.csv (1961-1990) and GWOHourly1991-2021.csv (1991-2021) into files for each observation station.

In [ ]:
## 1961-1990
GWO_stns2stn(fname=hourly_file_bf1990, db_path=db_path, stn_path=dir_hourly, hourly=True, overwrite=False)

In [ ]:
## 1991-
## Taking time to read CSV file.
GWO_stns2stn(fname=hourly_file_af1991, db_path=db_path, stn_path=dir_hourly, hourly=True, overwrite=False)